The usual magic stuff

In [1]:
import pythran
%load_ext pythran.magic

Test basic types

In [2]:
%%pythran
# simple types
#pythran export identity(int)
#pythran export identity(None)
#pythran export identity(str)

# parametric types
#pythran export identity(int list)
#pythran export identity(int set)
#pythran export identity(int:str dict)
#pythran export identity((int, int, str))

# numpy stuff
#pythran export identity(int[])
#pythran export identity(int[:,:])
#pythran export identity(int[][][])

def identity(x):
    return x

In [3]:
for elem in (int, str, list, set, dict):
    assert isinstance(identity(elem()), elem), elem

In [4]:
assert identity(None) is None
assert isinstance(identity((1,1,"1")), tuple)

Numpy arrays keep the same id when passed through pythran. this is not guaranteed for other containers

In [5]:
import numpy
x = numpy.ones(1, dtype=int)
assert x is identity(x)

y = numpy.ones((1, 1), dtype=int)
assert y is identity(y)

z = numpy.ones((1, 1, 1), dtype=int)
assert z is identity(z)

It's possible to declare the overloads in a single export using the ``or`` keyword

In [6]:
%%pythran
#pythran export strint(str or int, str or int)
def strint(x, y):
    return y, x

In [7]:
strint(1, 2)

(2, 1)

In [8]:
strint('1', '2')

('2', '1')

In [9]:
strint(1, '2')

('2', 1)

In [10]:
strint('1', 2)

(2, '1')

The ``or`` operator also works inside polymorphic types, but it has lower precedence than ``set``, ``dict`` etc.

In [11]:
%%pythran
#pythran export set_of(int or str set)
def set_of(x): return x

In [12]:
set_of(1)

1

In [13]:
set_of({'1'})

{'1'}

Use the ``[`` operator to force ordering

In [14]:
%%pythran
#pythran export set_of([int or str] set)
def set_of(x): return x

In [15]:
set_of({1})

{1}

In [16]:
set_of({'1'})

{'1'}

It's also possible to declare multiple entires in the same ``pythran export`` line

In [17]:
%%pythran
#pythran export foo(int), foo(str)
def foo(s): return s

In [18]:
foo(1), foo('1')

(1, '1')

The pythran export can also be used to export a global variable. But the global variable is not going to be shared, consider it as a read only view!

In [19]:
%%pythran
# pythran export thing
thing = 'stuff that matter'

In [20]:
thing

'stuff that matter'

It's also possible to ask pythran to export raw function pointer, using the ``capsule`` keyword.

In [21]:
%%pythran
#pythran export capsule corp(int, float)
def corp(x, y):
    return x + y

In [22]:
str(corp)[:40] + '...'

'<capsule object "corp(int, float)" at 0x...'

Pythran accept pointer type, but it's only meaningful inside a capsule

In [23]:
%%pythran
#pythran export capsule corp(int*, int)
def corp(data, size):
    return data[size/2]

A Pythran function can take a capsule as input, using function type signatures.

In [24]:
%%pythran
#pythran export higher_order(int(int), int)
def higher_order(f, val):
    return f(val)
#pythran export capsule dummy(int)
def dummy(n):
    return n + 1

In [25]:
higher_order(dummy, 3)

4